In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

qtInter,qtOut,qtInput,e,alpha,epoca = 100,10,784,0,0.1,0
dataset = pd.read_csv('mnist_train.csv')
v = np.random.randn(qtInput, qtInter) / np.sqrt(qtInput)
w = np.random.randn(qtInter, qtOut) / np.sqrt(qtInter)
ativacao = 'sig'
limiteLinhas = 60001
#Recomendado colocar 10000 pois o algoritmo está lento, e pode demorar para testar
#Leva em media 10-15 minutos para rodar
#Com 10000 linhas ele pega 87-89% de precisão
#Com todas 60000 chega proximo a 94%

In [2]:
grafico = np.array([[0]*2])
graficoTreino = np.array([[0]*2])

In [3]:
def export_weights(filenameV,filenameW):
    if (not os.path.exists(fV) and not os.path.exists(fW)):
        exW = pd.DataFrame(w)
        exV = pd.DataFrame(v)
        exW.to_csv('WCSV.csv', index = False, header = False)
        exV.to_csv('VCSV.csv', index = False, header = False) 
        
def import_weights():
    global v, w
    recV = pd.read_csv('VCSV.csv',sep=',',header = None,dtype=float)
    recW = pd.read_csv('WCSV.csv',sep=',',header = None,dtype=float)
    v = recV.values
    w = recW.values

In [4]:
#export_weights('VtesteCSV.csv','WtesteCSV.csv')
#import_weights()

In [5]:
def trataLinha(inp):
    retorno = np.array(inp)
    return retorno/255

def targetVetor(labels, num_classes=10):
    retTarget = np.eye(num_classes)[labels]
    return retTarget

def inputZ(inp):
    inX = np.zeros(qtInter)
    for j in range (qtInter):
        inX[j] = np.sum(inp * np.array(v[:,j]))
    return inX

def inputY(inp,pesos):
    inX = np.zeros(qtOut)
    for k in range (qtOut):
        inX[k] = np.sum(inp * np.array(pesos[:,k]))
    return inX

def deltainJ (dK,peso):
    inJ = np.zeros(qtInter)
    for j in range (qtInter):
        for k in range (qtOut):
            inJ[j] += dK[k] * peso[j][k]
    return inJ

def funcAtivacao(x,func) :
    if (func == 'relu') : 
        return np.maximum(x,0) 
    elif (func == 'sig') : 
        return (1/(1+np.exp(-x)))
    elif (func == 'tanh'):
        return np.tanh(x)

def derivada(x,func):        
    if (func == 'relu') : 
        return (0 if x<0 else 1)
    elif (func == 'sig') : 
        return (x * (1 - x))
    elif (func == 'tanh') :
        return (1 - np.power(x,2))

def verificaAcerto(tar,out):
    indTar = np.unravel_index(np.argmax(tar, axis=None), tar.shape)
    indOut = np.unravel_index(np.argmax(out, axis=None), out.shape)
    if (indTar == indOut):
        return True
    else :
        return False

def deltaK(targetK,Yk,func) :
    dK = np.zeros(qtOut)
    for i in range (qtOut) :
        erro = targetK[i] - Yk[i]
        deriv = derivada(Yk[i],func)
        dK[i] = erro * deriv
    return dK

def deltaJ(inJ,inZ,func):
    delJ = np.zeros(qtInter)
    for j in range (qtInter):
        deriv = derivada(inZ[j],func)
        delJ[j] = inJ[j] * deriv
    return delJ

def deltaW(dK,Ze):
    dW = np.empty_like(w)
    for j in range(qtInter):
        for k in range(qtOut):
            dW[j][k] = alpha * dK[k] * Ze[j]
    return dW

def deltaV(linha,dJ):
    dV = np.empty_like(v)
    for i in range(qtInput):
        for j in range (qtInter):
            dV[i][j] = alpha * dJ[j] * linha[i]
    return dV

In [6]:
def algoritmo(isTreino,qtEpoca):
    global epoca
    for i in range (qtEpoca):
        readLine(dataset,isTreino)
        epoca+=1

In [7]:
def readLine(dSet,isTreino):
    global e
    r,e = 1,0
    for row in dSet.itertuples(index=False):
        if (r<limiteLinhas):
            linha = list(row)
            target = targetVetor(linha.pop(0))
            linha = trataLinha(linha)
            forward(linha,isTreino,target,r)
            r+=1

In [8]:
def forward(linha,isTreino,target,r):
    
        global e
        global grafico
        global graficoTreino
        inZ = inputZ(linha)
        Z = funcAtivacao(inZ,ativacao)
        inY = inputY(Z,w)
        Y = funcAtivacao(inY,ativacao)
        if (not verificaAcerto(target,Y)):
            e+=1

        a = 100 - ((e/r)*100)
        if (r!=0 and r%100 == 0):
            #print (target)
            #print (Y)
            print (f'Linha: {r} Epoca: {epoca} Erros: {e}  Acerto: {a} %' + '\n' + '_________' )
        if isTreino:
            backPropagation(inZ,Z,inY,Y,target,linha)
            graficoTreino = np.append(graficoTreino,[[r,a]],axis=0)
        if (not isTreino) :
            grafico = np.append(grafico,[[r,a]],axis=0)

In [9]:
def backPropagation(inZ,Z,inY,Y,target,linha):
    delK,inJ,delJ =[],[],[]
    delK = deltaK(target,Y,ativacao) 
    inJ  = deltainJ(delK,w)
    delJ = deltaJ(inJ,Z,ativacao)
    atualizaPesos(delK,Z,linha,delJ)

In [10]:
def atualizaPesos(delK,Z,linha,delJ):
    global w
    global v
    correcaoW = deltaW(delK,Z)
    correcaoV = deltaV(linha,delJ)
    w = corrigePeso(w,correcaoW)
    v = corrigePeso(v,correcaoV)

In [11]:
def corrigePeso(peso,delta):
    return np.add(peso,delta)

In [12]:
def testar():
    global dataset
    dataset = pd.read_csv('mnist_test.csv')
    algoritmo(False,1)
    plt.plot(grafico[:,0],grafico[:,1])
    plt.xlabel('Linhas')
    plt.ylabel('Porcentagem de Acerto')
    plt.title('Teste da rede | Sigmoid *Teste*')
    plt.show()

In [13]:
def treinar():
    algoritmo(True,1)
    plt.plot(graficoTreino[:,0],graficoTreino[:,1])
    plt.xlabel('Linhas')
    plt.ylabel('Porcentagem de Acerto')
    plt.title('Aprendizado da rede | Sigmoid *Treino* ')
    plt.show()

In [14]:
treinar()

Linha: 100 Epoca: 0 Erros: 87  Acerto: 13.0 %
_________
Linha: 200 Epoca: 0 Erros: 166  Acerto: 17.0 %
_________
Linha: 300 Epoca: 0 Erros: 239  Acerto: 20.333333333333343 %
_________
Linha: 400 Epoca: 0 Erros: 300  Acerto: 25.0 %
_________
Linha: 500 Epoca: 0 Erros: 363  Acerto: 27.400000000000006 %
_________
Linha: 600 Epoca: 0 Erros: 425  Acerto: 29.166666666666657 %
_________
Linha: 700 Epoca: 0 Erros: 477  Acerto: 31.85714285714286 %
_________
Linha: 800 Epoca: 0 Erros: 526  Acerto: 34.25 %
_________
Linha: 900 Epoca: 0 Erros: 572  Acerto: 36.44444444444444 %
_________
Linha: 1000 Epoca: 0 Erros: 609  Acerto: 39.1 %
_________
Linha: 1100 Epoca: 0 Erros: 655  Acerto: 40.45454545454545 %
_________
Linha: 1200 Epoca: 0 Erros: 694  Acerto: 42.166666666666664 %
_________
Linha: 1300 Epoca: 0 Erros: 736  Acerto: 43.38461538461539 %
_________
Linha: 1400 Epoca: 0 Erros: 762  Acerto: 45.57142857142858 %
_________
Linha: 1500 Epoca: 0 Erros: 794  Acerto: 47.06666666666667 %
_________
Linha

Linha: 11900 Epoca: 0 Erros: 2175  Acerto: 81.72268907563026 %
_________
Linha: 12000 Epoca: 0 Erros: 2182  Acerto: 81.81666666666666 %
_________
Linha: 12100 Epoca: 0 Erros: 2195  Acerto: 81.85950413223141 %
_________
Linha: 12200 Epoca: 0 Erros: 2203  Acerto: 81.94262295081967 %
_________
Linha: 12300 Epoca: 0 Erros: 2216  Acerto: 81.98373983739837 %
_________
Linha: 12400 Epoca: 0 Erros: 2227  Acerto: 82.04032258064515 %
_________
Linha: 12500 Epoca: 0 Erros: 2241  Acerto: 82.072 %
_________
Linha: 12600 Epoca: 0 Erros: 2255  Acerto: 82.10317460317461 %
_________
Linha: 12700 Epoca: 0 Erros: 2270  Acerto: 82.1259842519685 %
_________
Linha: 12800 Epoca: 0 Erros: 2282  Acerto: 82.171875 %
_________
Linha: 12900 Epoca: 0 Erros: 2289  Acerto: 82.25581395348837 %
_________
Linha: 13000 Epoca: 0 Erros: 2305  Acerto: 82.26923076923077 %
_________
Linha: 13100 Epoca: 0 Erros: 2323  Acerto: 82.26717557251908 %
_________
Linha: 13200 Epoca: 0 Erros: 2339  Acerto: 82.28030303030303 %
________

Linha: 23300 Epoca: 0 Erros: 3286  Acerto: 85.8969957081545 %
_________
Linha: 23400 Epoca: 0 Erros: 3294  Acerto: 85.92307692307692 %
_________
Linha: 23500 Epoca: 0 Erros: 3303  Acerto: 85.94468085106382 %
_________
Linha: 23600 Epoca: 0 Erros: 3311  Acerto: 85.97033898305085 %
_________
Linha: 23700 Epoca: 0 Erros: 3319  Acerto: 85.9957805907173 %
_________
Linha: 23800 Epoca: 0 Erros: 3328  Acerto: 86.01680672268907 %
_________
Linha: 23900 Epoca: 0 Erros: 3339  Acerto: 86.02928870292887 %
_________
Linha: 24000 Epoca: 0 Erros: 3347  Acerto: 86.05416666666667 %
_________
Linha: 24100 Epoca: 0 Erros: 3354  Acerto: 86.08298755186722 %
_________
Linha: 24200 Epoca: 0 Erros: 3360  Acerto: 86.11570247933884 %
_________
Linha: 24300 Epoca: 0 Erros: 3369  Acerto: 86.1358024691358 %
_________
Linha: 24400 Epoca: 0 Erros: 3379  Acerto: 86.15163934426229 %
_________
Linha: 24500 Epoca: 0 Erros: 3389  Acerto: 86.16734693877551 %
_________
Linha: 24600 Epoca: 0 Erros: 3398  Acerto: 86.18699186

KeyboardInterrupt: 

In [ ]:
testar()